In [41]:
import torch
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import sys
import os
from pathlib import Path

current_dir = Path(os.getcwd())
if current_dir.name == 'model_components':
    parent_dir = str(current_dir.parent)
else:
    parent_dir = str(current_dir)

if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from model_components.generator import Generator

MODEL_PATH = current_dir.parent / "models" / "srresnet_mse.pth"
IMAGE_PATH = current_dir.parent / "comic.png"
UPSCALE_FACTOR = 4

# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Initialize the generator
generator = Generator().to(device)

# Load weights
print(f"Loading model from {MODEL_PATH}...")
checkpoint = torch.load(MODEL_PATH, map_location=device)
if 'generator_state_dict' in checkpoint:
    generator.load_state_dict(checkpoint['generator_state_dict'])
    print("Generator loaded successfully.")
else:
    generator.load_state_dict(checkpoint) 

# [cite_start]"During test time we turn batch-normalization update off" [cite: 245]
generator.eval()

GAN_NO_PIXEL_MODEL_PATH = current_dir.parent / "models" / "best_model_without_pixel.pth"

# Initialize the generator
srgan_no_pixel_generator = Generator().to(device)

# Load weights
print(f"Loading model from {GAN_NO_PIXEL_MODEL_PATH}...")
checkpoint = torch.load(GAN_NO_PIXEL_MODEL_PATH, map_location=device)

srgan_no_pixel_generator.load_state_dict(checkpoint['generator_state_dict'])
print("Generator loaded successfully.")


GAN_MODEL_PATH = current_dir.parent / "models" / "best_model.pth"

# Initialize the generator
srgan_generator = Generator().to(device)

# Load weights
print(f"Loading model from {GAN_MODEL_PATH}...")
checkpoint = torch.load(GAN_MODEL_PATH, map_location=device)

srgan_generator.load_state_dict(checkpoint['generator_state_dict'])
print("Generator loaded successfully.")


# [cite_start]"During test time we turn batch-normalization update off" [cite: 245]
srgan_generator.eval()

Using device: cuda
Loading model from c:\Users\yingq\Desktop\SRGAN\SRGAN-Implementation\models\srresnet_mse.pth...
Loading model from c:\Users\yingq\Desktop\SRGAN\SRGAN-Implementation\models\best_model_without_pixel.pth...
Generator loaded successfully.
Loading model from c:\Users\yingq\Desktop\SRGAN\SRGAN-Implementation\models\best_model.pth...
Generator loaded successfully.


Generator(
  (conv1): Conv2d(3, 64, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
  (prelu): PReLU(num_parameters=1)
  (res_blocks): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (prelu): PReLU(num_parameters=1)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (prelu): PReLU(num_parameters=1)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, 

In [42]:
# Load the High-Resolution (HR) image
hr_image_pil = Image.open(IMAGE_PATH).convert("RGB")

# "Blur It" (Create the Low-Resolution LR image)
# We use bicubic downsampling, as done in the paper's data pipeline
w, h = hr_image_pil.size
lr_w, lr_h = w // UPSCALE_FACTOR, h // UPSCALE_FACTOR
lr_image_pil = hr_image_pil.resize((lr_w, lr_h), Image.BICUBIC)

bicubic_image_pil = lr_image_pil.resize((w, h), Image.BICUBIC)

lr_transform = transforms.ToTensor()
lr_tensor = lr_transform(lr_image_pil).unsqueeze(0).to(device)

print(f"Original HR image size: {hr_image_pil.size}")
print(f"LR image size: {lr_image_pil.size}")
print(f"LR tensor shape for model: {lr_tensor.shape}")

Original HR image size: (250, 361)
LR image size: (62, 90)
LR tensor shape for model: torch.Size([1, 3, 90, 62])


In [ ]:
# Utility function to display images    
def display_image(tensor, title=""):
    img = tensor.cpu().permute(1, 2, 0)
    # Clip values to [0, 1] for proper matplotlib display
    img = torch.clamp(img, 0, 1)
    plt.imshow(img)
    plt.title(title)
    plt.axis("off")

with torch.no_grad():
    sr_tensor_tanh = generator(lr_tensor)
    sr_tensor_tanh_gan = srgan_generator(lr_tensor)
    sr_tensor_tanh_gan_no_pixel = srgan_no_pixel_generator(lr_tensor)


# Rescale from [-1, 1] (tanh output) to [0, 1]
sr_tensor = (sr_tensor_tanh + 1.0) / 2.0
sr_tensor_gan = (sr_tensor_tanh_gan + 1.0) / 2.0

sr_tensor_gan_no_pixel = (sr_tensor_tanh_gan_no_pixel + 1.0) / 2.0

sr_tensor = sr_tensor.squeeze(0)
sr_tensor_gan = sr_tensor_gan.squeeze(0)
sr_tensor_gan_no_pixel = sr_tensor_gan_no_pixel.squeeze(0)


bicubic_tensor = transforms.ToTensor()(bicubic_image_pil)
hr_tensor = transforms.ToTensor()(hr_image_pil)

plt.figure(figsize=(15, 5))

# Plot 1: Bicubic
plt.subplot(1, 5, 1)
display_image(bicubic_tensor, title=f"Bicubic Upscale (Blurry)")

plt.subplot(1, 5, 2)
display_image(sr_tensor, title=f"SRResNet Generated")


plt.subplot(1, 5, 3)
display_image(sr_tensor_gan_no_pixel, title=f"SRGAN Generated")

plt.subplot(1, 5, 4)
display_image(sr_tensor_gan, title=f"SRGAN Generated With Pixel Loss")

plt.subplot(1, 5, 5)
display_image(hr_tensor, title="Original High-Resolution")

plt.tight_layout()
plt.show()

# still tuning the model so the SRGAN generated here is different, I accidentally set the starting learning rate to 0.00001, surprisingly it works better than 0.0001.